<a href="https://colab.research.google.com/github/lagom-QB/M11/blob/master/EMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# EMNIST

EMNIST is an extention of MNIST dataset. It has 47 classes (handwritten digits and leters), some of the letters are represented as two classes (upper and lower case) and some (ex. o, s) has only one class associated with them.

In [0]:
from torchvision import datasets, transforms
import torch

train_dataset = datasets.EMNIST('/data', train=True, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]),
                                split="balanced")

test_dataset = datasets.EMNIST('../data', train=False, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]),
                                split="balanced")

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

#1

Find correspondence between classess and letters/digits

In [0]:
letters = train_dataset.classes_split_dict['letters']
digits  = train_dataset.classes_split_dict['digits']
byclass = train_dataset.classes_split_dict['byclass']  # if not why not
balanced = train_dataset.classes_split_dict['balanced']

In [5]:
len(balanced)

47

#2

Build and train a dense classifier for EMNIST dataset. Try to achieve the highest accuracy you can.\
Explain, what experiments have you perform.\
What optimizers have you tested?\
How network/optimizer parameters were chosen.

In [0]:
import torch.nn as nn
from torch.nn.modules import loss
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

class Net(nn.Module):
    
    def __init__(self):
        
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 64)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [0]:
learning_rate = 0.001 # Small enough
num_epochs = 25 #If not why not
num_classes = len(balanced)
batch_size = 100

In [0]:
model = Net()

device    = torch.device("cuda")
# device    = torch.device("cpu")

model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()

optimizer_1 = torch.optim.Adam(
                    model.parameters(), 
                    lr=learning_rate)  

optimizer = torch.optim.Adagrad(
                    model.parameters(),
                    lr = learning_rate, 
                    lr_decay = 0.01)
scheduler = torch.optim.lr_scheduler.StepLR(
                                            optimizer,
                                            0.5)

In [0]:
def test_ass(writer, model, test_loader, loss_function, device = device):
    model.eval()
    
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
    
            output = model(data)
    
            test_loss += loss_function(output, target).sum().item()
    
            pred = output.argmax(dim=1, keepdim=True)
    
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    accuracy = 100. * correct / len(test_loader.dataset)
    
    writer.add_scalar("test_loss", test_loss, global_step=epoch)
    writer.add_scalar("accuracy", accuracy, global_step=epoch)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
def train_ass(writer, model, train_loader, optimizer, loss_function, epoch , device = device):
    model.train()

    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)

        optimizer.zero_grad()

        output = model(data)
        loss = loss_function(output, target)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    train_loss /= len(train_loader.dataset)
    writer.add_scalar("train_loss", train_loss, global_step=epoch)

In [0]:
writer_2_i = SummaryWriter(log_dir=f"log_2i")
for epoch in range(20):
    train_ass(writer_2_i, model, train_loader, optimizer, loss.CrossEntropyLoss(), epoch)    
    test_ass(writer_2_i, model, test_loader, loss.CrossEntropyLoss(), device = device)

Train Epoch: 0 [0/112800 (0%)]	Loss: 0.843950
Train Epoch: 0 [32000/112800 (28%)]	Loss: 1.076501
Train Epoch: 0 [64000/112800 (57%)]	Loss: 0.940363
Train Epoch: 0 [96000/112800 (85%)]	Loss: 1.121796

Test set: Average loss: 0.0152, Accuracy: 13512/18800 (72%)

Train Epoch: 1 [0/112800 (0%)]	Loss: 1.107953
Train Epoch: 1 [32000/112800 (28%)]	Loss: 0.964005
Train Epoch: 1 [64000/112800 (57%)]	Loss: 1.078275
Train Epoch: 1 [96000/112800 (85%)]	Loss: 0.859198

Test set: Average loss: 0.0151, Accuracy: 13547/18800 (72%)

Train Epoch: 2 [0/112800 (0%)]	Loss: 1.063351
Train Epoch: 2 [32000/112800 (28%)]	Loss: 1.262464
Train Epoch: 2 [64000/112800 (57%)]	Loss: 1.172318
Train Epoch: 2 [96000/112800 (85%)]	Loss: 1.200750

Test set: Average loss: 0.0151, Accuracy: 13560/18800 (72%)

Train Epoch: 3 [0/112800 (0%)]	Loss: 1.060959
Train Epoch: 3 [32000/112800 (28%)]	Loss: 0.994583
Train Epoch: 3 [64000/112800 (57%)]	Loss: 0.899548
Train Epoch: 3 [96000/112800 (85%)]	Loss: 1.003954

Test set: Average

In [0]:
writer_2_ii = SummaryWriter(log_dir=f"log_2ii")
for epoch in range(20):
    train_ass(writer_2_ii, model, train_loader, optimizer, loss.CrossEntropyLoss(), epoch)    
    test_ass(writer_2_ii, model, test_loader, loss.CrossEntropyLoss(), device = device)

Adam, lr (0.01) >> Accuracy of 72%\
Adagrad, lr (0.01) >> Accuracy of 73%\
 .

#3

Train the same classifier on MNIST dataset (you will have to replace the last dense layer of your model)

*MNIST*

In [0]:
from torchvision import datasets, transforms
import torch

train_dataset_mnist = datasets.MNIST(
                                '/data', 
                                train = True, 
                                download = True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
test_dataset_mnist = datasets.MNIST(
                                '../data', 
                                train = False, 
                                download = True,
                                transform = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
train_loader_mnist = torch.utils.data.DataLoader(
                                        train_dataset_mnist, 
                                        batch_size=64, 
                                        shuffle=True)

test_loader_mnist = torch.utils.data.DataLoader(
                                        test_dataset_mnist, 
                                        batch_size=64, 
                                        shuffle=True)

In [0]:
class MNISTNet(nn.Module):
    
    def __init__(self):
        
        super(MNISTNet, self).__init__()

        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10) # changed from self.fc2 = nn.Linear(1000, 64)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

model_mnist = MNISTNet()

In [0]:
model_mnist = model.to(device)

optimizer_mnist_1 = torch.optim.Adam(
                    model_mnist.parameters(), 
                    lr=learning_rate)  

optimizer_mnist = torch.optim.Adagrad(
                    model_mnist.parameters(),
                    lr = learning_rate, 
                    lr_decay = 0.01)

scheduler = torch.optim.lr_scheduler.StepLR(
                                            optimizer_mnist,
                                            0.5)

writer_3_i = SummaryWriter(log_dir=f"log_3i")
for epoch in range(20):
    train_ass(writer_3_i, model_mnist, train_loader_mnist, optimizer_mnist, loss.CrossEntropyLoss(), epoch)    
    test_ass(writer_3_i, model_mnist, test_loader_mnist, loss.CrossEntropyLoss(), device = device)

In [0]:
writer_3_ii = SummaryWriter(log_dir=f"log_3ii")
for epoch in range(20):
    train_ass(writer_3_ii, model_mnist, train_loader_mnist, optimizer_mnist_1, loss.CrossEntropyLoss(), epoch)    
    test_ass(writer_3_ii, model_mnist, test_loader_mnist, loss.CrossEntropyLoss(), device = device)

Adagrad(96%)\
Adam(99%)


#4

Use EMNIST classifier to classify MNIST test dataset.\

* To do this you'll need to restrict last layer to the labels which correspond to digits and choose the largest value 

(\
    torch tensors can be indexed with lists, like this:\
    --> torch.tensor(
        [
            [1,2],
            [3,4], 
            [5, 6]
        ])[[0, 1]]\
).\
\
Which of the models performs better?

In [0]:
def test_ass_2(writer, model, test_loader, loss_function, device = device):
    model.eval()
    
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
    
            output = model(data)
    
            test_loss += loss_function(output[:,0:10], target).sum().item()
    
            pred = output[:,0:10].argmax(dim=1, keepdim=True)
    
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    writer.add_scalar("test_loss", test_loss, global_step=epoch)
    writer.add_scalar("accuracy", accuracy, global_step=epoch)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
def train_ass_2(writer, model, train_loader, optimizer, loss_function, epoch , device = device):
    model.train()

    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)

        optimizer.zero_grad()

        output = model(data)
        loss = loss_function(output[:,0:10], target)
        
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
    train_loss /= len(train_loader.dataset)

    writer.add_scalar("train_loss", train_loss, global_step=epoch)

for epoch in range(20):
    train_ass_2(model, train_loader_mnist, optimizer, loss.CrossEntropyLoss(), epoch)    
    test_ass_2(model, test_loader_mnist, loss.CrossEntropyLoss(), device = device)

In [0]:
writer_4_i = SummaryWriter(log_dir=f"log_4i")

for epoch in range(20):
    train_ass_2(writer_4_i, model, train_loader_mnist, optimizer, loss.CrossEntropyLoss(), epoch)    
    test_ass_2(writer_4_i, model, test_loader_mnist, loss.CrossEntropyLoss(), device = device)

In [0]:
writer_4_ii = SummaryWriter(log_dir=f"log_4ii")

for epoch in range(20):
    train_ass_2(writer_4_ii, model, train_loader_mnist, optimizer, loss.CrossEntropyLoss(), epoch)    
    test_ass_2(writer_4_ii, model, test_loader_mnist, loss.CrossEntropyLoss(), device = device)

Adam(99%)\
Adagrad(99%)

In [0]:
%tensorboard --logdir log_2i

In [0]:
%tensorboard --logdir log_2ii

In [0]:
%tensorboard --logdir log_3i

In [0]:
%tensorboard --logdir log_3ii

In [0]:
%tensorboard --logdir log_4i

In [0]:
%tensorboard --logdir log_4ii

EMNIST is an extention of MNIST dataset.
It has 47 classes (handwritten digits and leters), some of the letters are represented as two classes (upper and lower case) and some (ex. o, s) has only one class associated with them.

1. Find correspondence between classess and letters/digits.
2. Build and train a dense classifier for EMNIST dataset. Try to achieve the highest accuracy you can. Explain, what experiments have you perform. What optimizers have you tested? How network/optimizer parameters were chosen.
3. Train the same classifier on MNIST dataset (you will have to replace the last dense layer of your model).
4. Use EMNIST classifier to classify MNIST test dataset. To do this you'll need to restrict last layer to the labels which correspond to digits and choose the largest value (torch tensors can be indexed with lists, like this: `torch.tensor([[1,2],[3,4], [5, 6]])[[0, 1]]`). Which of the models performs better?
5. Present a short (2-3 pages long) report on this experiment. What approaches have you tried? How can you explain the results you got? Include tensorboard plots in the experiment report.
